In [79]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import google.generativeai as genai
from langchain.chains.question_answering import load_qa_chain


In [21]:
from dotenv import load_dotenv
load_dotenv()
import os
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))



In [22]:
import os

In [23]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [24]:
doc=read_doc('documents/')
len(doc)

58

In [25]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [26]:
documents=chunk_data(docs=doc)
len(documents)
# print("documents", documents)

58

In [32]:
## Embedding Technique Of OPENAI
# embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
# embeddings
# embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B6A744D900>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [75]:
vectors=embeddings.embed_query("How are you?")
len(vectors)
# print("vectors", vectors)

768

In [73]:
pinecone.Pinecone(
api_key = '5e17a874-024c-4c0b-a725-3296e6c7f28c',
)
index_name = "langchain"
os.environ['PINECONE_API_KEY'] = '5e17a874-024c-4c0b-a725-3296e6c7f28c'



In [76]:
from langchain_pinecone import PineconeVectorStore
vectorstore_from_docs = PineconeVectorStore.from_documents(
    doc,
    index_name=index_name,
    embedding=embeddings
)

In [77]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=vectorstore_from_docs.similarity_search(query,k=k)
    return matching_results

In [18]:
from langchain.chains.question_answering import load_qa_chain
# from langchain import OpenAI

In [80]:
# llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")
chain=load_qa_chain(llm,chain_type="stuff")

In [81]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [82]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  \nFisheries \n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana \nwith targeted investment of ` 6,000 crore to further enable activities of \nfishermen, fish vendors, and micro & small enterprises, improve value chain \nefficiencies, and expand the market. \nCooperation \n25. For farmers, especially small and marginal farmers, and other \nmarginalised sections, the government is promoting cooperative-based \neconomic development model. A new Ministry of Coo

c:\Users\Rahul\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


The agriculture credit target will be increased to ` 20 lakh crore.
